<a href="https://colab.research.google.com/github/Astromax/Retina_Disease_Classification/blob/main/EyeDiseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this is to determine whether or not a retina is healthy, and, if not, which of six different diseases it has.

In [3]:
#First let's mount the Drive so we can access the data...
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os
os.chdir('/content/drive/MyDrive/EyeDiseaseDetection/')

In [6]:
from resnet_module_helpers_tf import *

# Display
from IPython.display import Image, display
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from plotly.subplots import make_subplots
from sklearn.metrics import multilabel_confusion_matrix as MLCM
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.activations import relu
from keras.optimizers import Adam
from keras.optimizers.schedules import ExponentialDecay
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout, Input, BatchNormalization, Activation
from keras.layers.experimental.preprocessing import RandomCrop, RandomFlip, RandomRotation
from tqdm import tqdm

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [19]:
#Establish location of data
data_dir = './Data/Data/train'
image_dir = data_dir+'/train'

In [20]:
filenames = os.listdir(image_dir)
print('Number of files: ', len(filenames))

Number of files:  3435


In [ ]:
#Let's check for anomalies and see how many of each disease type there are....
labels_df = pd.read_csv(data_dir+'/train.csv')
print('Number of all-zero anomalies: ', labels_df[(labels_df['opacity'] == 0) & (labels_df['diabetic retinopathy']==0)
& (labels_df['glaucoma'] == 0) & (labels_df['macular edema'] == 0) & (labels_df['macular degeneration'] == 0) 
& (labels_df['retinal vascular occlusion'] == 0) & (labels_df['normal'] == 0)].count())

print('Number of inconsistencies: ', labels_df[((labels_df['opacity'] == 1) | (labels_df['diabetic retinopathy']==1)
| (labels_df['glaucoma'] == 1) | (labels_df['macular edema'] == 1) | (labels_df['macular degeneration'] == 1) 
| (labels_df['retinal vascular occlusion'] == 1)) & (labels_df['normal'] == 1)].count())

print('Opacity count: ')
print(labels_df['opacity'].value_counts())
print('Diabetic Retinopathy count: ')
print(labels_df['diabetic retinopathy'].value_counts())
print('Glaucoma count: ')
print(labels_df['glaucoma'].value_counts())
print('Macular Edema count:')
print(labels_df['macular edema'].value_counts())
print('Macular Degeneration count:')
print(labels_df['macular degeneration'].value_counts())
print('Retinal Vascular Occlusion count: ')
print(labels_df['retinal vascular occlusion'].value_counts())
print('Normal count: ')
print(labels_df['normal'].value_counts())